This file is for the assignment 4 of Monte Carlo Method at University Paris Dauphine master 2 MASEF

Author: Yu Xiang

Contact: shawnxiangyu@yahoo.com

In [63]:
# import libaries and set the setting for plot
%matplotlib inline
import numpy as np
from scipy import linalg
from scipy.stats import norm
import time
import matplotlib.pyplot as plt
import math


# choose a large font size by default and use tex for math
fontsize = 10
params = {'axes.labelsize': fontsize + 2,
      'font.size': fontsize + 2,
      'legend.fontsize': fontsize + 2,
      'xtick.labelsize': fontsize,
      'ytick.labelsize': fontsize}
plt.rcParams.update(params)

In [64]:
# simulate based on Euler methods
def sim_Euler(T=1, nr_step=100, X0=100, r=0, sigma=0.15, nr_sims=10000):  
    #  We simulate with Euler scheme  

    dt = T / nr_step    
    Xs = np.zeros((nr_sims, nr_step + 1))
    Xs[:,0] = X0  
    for i in range(nr_step): 
        Xs[:,i+1] = Xs[:,i] +  r * Xs[:,i] * dt + sigma * Xs[:,i] * dt ** 0.5 * np.random.randn(1, nr_sims)  
    return Xs 



In [65]:
# test the function works
Xs = sim_Euler()

U = 130 
U_id = np.max(Xs, axis=1) - U < 0
K = 90
XT = Xs[:,-1]
r = 0 
T = 1
p = np.mean(np.exp(- r * T) * np.maximum(XT - K, 0) *  U_id)
p

9.527114240264039

In [66]:
def p_Euler_sim(T=1, nr_steps=1000, X0=100, r=0, sigma=0.15, nr_sims=10000, U=130, K=90,case='sim'):
    
       
    if case == 'sim': 
        nr_cases = len(nr_sims)
    elif case == 'step': 
        nr_cases = len(nr_steps)
        
    
    value_conf_interval = np.zeros((nr_cases, 5))
    
    for i in range(nr_cases):
        
        if case == 'sim': 
            nr_paths = nr_sims[i]
            nr_stepi = nr_steps
            nr = nr_paths
        elif case == 'step': 
            nr_stepi = nr_steps[i]
            nr_paths = nr_sims
            nr = nr_stepi
            
        Xs = sim_Euler(nr_step=nr_stepi, nr_sims=nr_paths)
        U_id = np.max(Xs, axis=1) - U < 0
        XT = Xs[:,-1]
        
        p = np.mean(np.exp(- r * T) * np.maximum(XT - K, 0) *  U_id)
     
    
        var_n = nr_paths / (nr_paths - 1) \
            * (np.mean((np.exp(- r * T) * np.maximum(XT - K, 0) *  U_id) ** 2) - p **2 ) 
    
        std_n = var_n ** 0.5

        # 95% interval corresponds to 1.96 * std_n / (sqrt(n))
        half_int = 1.96 * std_n / nr_paths ** 0.5 

        conf_interval = np.round([p - half_int,  p + half_int],4)      
        value_conf_interval[i, 0] = nr
        value_conf_interval[i, 1] = p
        
        value_conf_interval[i, 2:4] = conf_interval
        value_conf_interval[i, 4] = half_int * 2
        
    return value_conf_interval  

In [67]:
nr_sims = [1000, 5000, 10000, 50000, 100000]


value_conf_interval = p_Euler_sim(nr_sims=nr_sims)


import pandas as pd
result = pd.DataFrame(value_conf_interval,
                      columns=['Nr_sims', 'numerical_price', 
                               '95% conf_lower_bound', '95% conf_upper_bound','interval_length'])

result



,Nr_sims,numerical_price,95% conf_lower_bound,95% conf_upper_bound,interval_length
0,1000.0,9.472867,8.8421,10.1036,1.261545
1,5000.0,9.265224,8.9927,9.5378,0.545133
2,10000.0,9.350635,9.1542,9.5471,0.392963
3,50000.0,9.314344,9.2272,9.4015,0.174261
4,100000.0,9.293335,9.2317,9.3549,0.123221


### Comment: 
When the number of simulation increases, the confidence interval narrows. 

We overestimate the value of barrier option. 

In [70]:
nr_steps = [10, 50, 100, 1000, 5000, 10000, 20000]
value_conf_interval = p_Euler_sim(nr_sims=10000, nr_steps=nr_steps, case='step')


import pandas as pd
result = pd.DataFrame(value_conf_interval,
                      columns=['Nr_steps', 'numerical_price', 
                               '95% conf_lower_bound', '95% conf_upper_bound', 'interval_length'])
print(result)

   Nr_steps  numerical_price  95% conf_lower_bound  95% conf_upper_bound  \
0      10.0        10.149809                9.9442               10.3554   
1      50.0         9.467716                9.2688                9.6666   
2     100.0         9.569753                9.3720                9.7676   
3    1000.0         9.354784                9.1601                9.5495   
4    5000.0         9.321876                9.1271                9.5166   
5   10000.0         9.206553                9.0146                9.3985   
6   20000.0         9.354953                9.1609                9.5490   

   interval_length  
0         0.411268  
1         0.397753  
2         0.395602  
3         0.389336  
4         0.389495  
5         0.383869  
6         0.388160  


### Comment: 
When the number of step increases, the confidence interval length remains more or less the same, whowever the numerical value gets closer to the true value. 

#### Brownian Bridge Approach

In [71]:
x = 1
x

1